In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from matplotlib_venn import venn2 # install with pip install matplotlib-venn

from eurostatapiclient import EurostatAPIClient # eurostat API

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import dataproject as dp

In [2]:
# a. initializing the Eurostat API client
client = EurostatAPIClient('v2.1','json','en')

# Importing air passenger transport

In [3]:
# a. defining parameters for the Air Passenger transport dataset, refer to Eurostat for parameters
params = [('tra_meas','PAS_BRD'),('tra_cov','TOTAL'),('schedule','TOT'),('unit','PAS')]

# b. time parameters must be added individually (!!! SHOULD BE MADE AS A FUNCTION !!!)
year_start, year_end = 1998, 2020

time_param = []

for n in range(year_end-year_start+1):
    year = year_end - n
    time_param.append(('time',str(year)))
    
params += time_param
    
# c. importing the dataset
dataset = client.get_dataset('avia_paoc', params=params)
print(f'The dataset title is:\n{dataset.label}\n')

# d. using the API to turn it into a df and inspecting the head
air_pas = dataset.to_dataframe()
air_pas.head()

The dataset title is:
Air passenger transport by reporting country



,values,unit,tra_meas,tra_cov,schedule,geo,time
0,13208221.0,PAS,PAS_BRD,TOTAL,TOT,AT,1998
1,13778419.0,PAS,PAS_BRD,TOTAL,TOT,AT,1999
2,14766021.0,PAS,PAS_BRD,TOTAL,TOT,AT,2000
3,14697777.0,PAS,PAS_BRD,TOTAL,TOT,AT,2001
4,15195431.0,PAS,PAS_BRD,TOTAL,TOT,AT,2002


There are obviously some columns we don't want. More specifically the columns unit, tra_meas, tra_cov and schedule.

In [4]:
# a. Removing unwanted columns (!!! SHOULD BE MADE AS A FUNCTION !!!)
drop_these = ['unit','tra_meas','tra_cov','schedule']
air_pas.drop(drop_these, axis=1, inplace=True) # axis = 1 -> columns, inplace=True -> changed, no copy made
air_pas.head()

,values,geo,time
0,13208221.0,AT,1998
1,13778419.0,AT,1999
2,14766021.0,AT,2000
3,14697777.0,AT,2001
4,15195431.0,AT,2002


In [5]:
# b. Renaming values to passengers
air_pas.rename(columns = {'values':'passengers'}, inplace=True)

# c. and moving the column as the last, for a more intuitive layout (!!! SHOULD BE MADE AS A FUNCTION !!!)
move_col = air_pas.pop('passengers')
air_pas.insert(2, 'passengers', move_col)

air_pas.head()

,geo,time,passengers
0,AT,1998,13208221.0
1,AT,1999,13778419.0
2,AT,2000,14766021.0
3,AT,2001,14697777.0
4,AT,2002,15195431.0


In [6]:
geo_list = []

for g in air_pas['geo']:
    if g not in geo_list:
        geo_list.append(g)
        
print(geo_list)

['AT', 'BE', 'BG', 'CH', 'CY', 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'EU27_2007', 'EU27_2020', 'EU28', 'FI', 'FR', 'HR', 'HU', 'IE', 'IS', 'IT', 'LT', 'LU', 'LV', 'ME', 'MK', 'MT', 'NL', 'NO', 'PL', 'PT', 'RO', 'RS', 'SE', 'SI', 'SK', 'TR', 'UK']


In [7]:
#drop rows with EU27_2007 and EU27_2020 - check lige datasettet på eurostats hjemmeside
#måske lav en funktion der sletter alt Non-eu

In [8]:
def code_to_name(code): # kan den laves i to funktioner? En der laver code to name og en der sorterer unwanted fra?
    """ Takes a country code as input and returns the country name """
    code_dic = {
        'EU28' : 'European Union (28)',
        'BE' : 'Belgium',
        'BG' : 'Bulgaria',
        'CZ' : 'Czechia',
        'DK' : 'Denmark',
        'DE' : 'Germany',
        'EE' : 'Estonia',
        'IE' : 'Ireland',
        'EL' : 'Greece',
        'ES' : 'Spain',
        'FR' : 'France',
        'HR' : 'Croatia',
        'IT' : 'Italy',
        'CY' : 'Cyprus',
        'LV' : 'Latvia',
        'LT' : 'Lithuania',
        'LU' : 'Luxembourg',
        'HU' : 'Hungary',
        'MT' : 'Malta',
        'NL' : 'Netherlands',
        'AT' : 'Austria',
        'PL' : 'Poland',
        'PT' : 'Portugal',
        'RO' : 'Romania',
        'SI' : 'Slovenia',
        'SK' : 'Slovakia',
        'FI' : 'Finland',
        'SE' : 'Sweden',
        'UK' : 'United Kingdom'
    }
    
    if code in code_dic:
        return code_dic[code]
    else:
        return 'unwanted'


In [9]:
geo_names = []

for item in geo_list:
    name = code_to_name(item)
    if name != 'unwanted':
        geo_names.append(name)

print(geo_names)
len(geo_names)

['Austria', 'Belgium', 'Bulgaria', 'Cyprus', 'Czechia', 'Germany', 'Denmark', 'Estonia', 'Greece', 'Spain', 'European Union (28)', 'Finland', 'France', 'Croatia', 'Hungary', 'Ireland', 'Italy', 'Lithuania', 'Luxembourg', 'Latvia', 'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Sweden', 'Slovenia', 'Slovakia', 'United Kingdom']


29

In [10]:
air_pas['country_name'] = air_pas['geo'].apply(code_to_name)
air_pas.loc[20:25,('geo','country_name')]

,geo,country_name
20,AT,Austria
21,AT,Austria
22,AT,Austria
23,BE,Belgium
24,BE,Belgium
25,BE,Belgium


In [11]:
# Removing unwanted rows
I = air_pas.country_name.str.contains('unwanted')
air_pas.loc[I,:]

air_pas = air_pas.loc[I == False]
air_pas.reset_index(inplace = True)
air_pas.drop('index', axis=1, inplace=True)
air_pas #notice the output, now fewer rows

,geo,time,passengers,country_name
0,AT,1998,13208221.0,Austria
1,AT,1999,13778419.0,Austria
2,AT,2000,14766021.0,Austria
3,AT,2001,14697777.0,Austria
4,AT,2002,15195431.0,Austria
...,...,...,...,...
662,UK,2016,249076212.0,United Kingdom
663,UK,2017,264867264.0,United Kingdom
664,UK,2018,272386344.0,United Kingdom
665,UK,2019,277548891.0,United Kingdom


In [12]:
# Moving and renaming in one step
column = air_pas.pop('country_name')
air_pas.insert(0,'geo_name',column)

air_pas

,geo_name,geo,time,passengers
0,Austria,AT,1998,13208221.0
1,Austria,AT,1999,13778419.0
2,Austria,AT,2000,14766021.0
3,Austria,AT,2001,14697777.0
4,Austria,AT,2002,15195431.0
...,...,...,...,...
662,United Kingdom,UK,2016,249076212.0
663,United Kingdom,UK,2017,264867264.0
664,United Kingdom,UK,2018,272386344.0
665,United Kingdom,UK,2019,277548891.0


# Importing real GDP pr capita

In [13]:
# unit in chain linked volumes, ie. constant in terms of inflation and prices
params = [('unit','CLV10_EUR_HAB'),('na_item','B1GQ')] + time_param # refer to https://ec.europa.eu/eurostat/databrowser/view/NAMA_10_PC__custom_767670/default/table?lang=en
dataset = client.get_dataset('nama_10_pc',params=params)
print(f'The dataset title is:\n{dataset.label}\n')

The dataset title is:
Main GDP aggregates per capita



In [14]:
real_gdp = dataset.to_dataframe()
real_gdp.head()

,values,unit,na_item,geo,time
0,29750.0,CLV10_EUR_HAB,B1GQ,AT,1998
1,30750.0,CLV10_EUR_HAB,B1GQ,AT,1999
2,31710.0,CLV10_EUR_HAB,B1GQ,AT,2000
3,31990.0,CLV10_EUR_HAB,B1GQ,AT,2001
4,32360.0,CLV10_EUR_HAB,B1GQ,AT,2002


In [15]:
# a. removing columns unit and na_item
real_gdp.drop(['unit','na_item'], axis=1, inplace=True)

In [16]:
# b. moving and renaming values
column = real_gdp.pop('values')
real_gdp.insert(2,'eur',column)

In [17]:
# c. adding column with country name
real_gdp['geo_name'] = real_gdp['geo'].apply(code_to_name)

In [18]:
# d. remove 'unwanted' rows
I = real_gdp.geo_name.str.contains('unwanted')
real_gdp = real_gdp.loc[I == False]

In [19]:
# e. moving geo_name as first column
column = real_gdp.pop('geo_name')
real_gdp.insert(0,'geo_name',column)

real_gdp

,geo_name,geo,time,eur
0,Austria,AT,1998,29750.0
1,Austria,AT,1999,30750.0
2,Austria,AT,2000,31710.0
3,Austria,AT,2001,31990.0
4,Austria,AT,2002,32360.0
...,...,...,...,...
938,United Kingdom,UK,2016,32060.0
939,United Kingdom,UK,2017,32430.0
940,United Kingdom,UK,2018,32640.0
941,United Kingdom,UK,2019,32910.0


# Importing population data

In [20]:
# sex both male and female (T) and all age groups (TOTAL)
params = [('sex','T'),('age','TOTAL')] + time_param # refer to https://ec.europa.eu/eurostat/databrowser/view/NAMA_10_PC__custom_767670/default/table?lang=en
dataset = client.get_dataset('demo_pjan',params=params)
print(f'The dataset title is:\n{dataset.label}\n')

The dataset title is:
Population on 1 January by age and sex



In [21]:
pop = dataset.to_dataframe()
pop

,values,unit,age,sex,geo,time
0,65306.0,NR,TOTAL,T,AD,1998
1,65877.0,NR,TOTAL,T,AD,1999
2,65971.0,NR,TOTAL,T,AD,2000
3,65844.0,NR,TOTAL,T,AD,2001
4,66334.0,NR,TOTAL,T,AD,2002
...,...,...,...,...,...,...
1352,1771604.0,NR,TOTAL,T,XK,2016
1353,1783531.0,NR,TOTAL,T,XK,2017
1354,1798506.0,NR,TOTAL,T,XK,2018
1355,1795666.0,NR,TOTAL,T,XK,2019


In [22]:
# a. removing columns unit, age, sex
drop_these = ['unit','age','sex']
pop.drop(drop_these, axis=1, inplace=True)

In [23]:
# b. adding column with country name
pop['geo_name'] = pop['geo'].apply(code_to_name)

In [24]:
# c. removing unwanted rows
I = pop.geo_name.str.contains('unwanted')
pop = pop.loc[I == False]

In [25]:
# d. moving geo_name and geo as first two columns
column = pop.pop('geo')
pop.insert(0,'geo',column)
column = pop.pop('geo_name')
pop.insert(0,'geo_name',column)

# Merging the dataframes

In [44]:
# a. merging air_pas and real_gdp
merged = pd.merge(real_gdp, air_pas, on=['geo','geo_name','time'], how='left')

# b. merge with pop
full = pd.merge(merged, pop, on=['geo','geo_name','time'], how='left')

In [45]:
full

,geo_name,geo,time,eur,passengers,values
0,Austria,AT,1998,29750.0,13208221.0,7971116.0
1,Austria,AT,1999,30750.0,13778419.0,7982461.0
2,Austria,AT,2000,31710.0,14766021.0,8002186.0
3,Austria,AT,2001,31990.0,14697777.0,8020946.0
4,Austria,AT,2002,32360.0,15195431.0,8063640.0
...,...,...,...,...,...,...
662,United Kingdom,UK,2016,32060.0,249076212.0,65379044.0
663,United Kingdom,UK,2017,32430.0,264867264.0,65844142.0
664,United Kingdom,UK,2018,32640.0,272386344.0,66273576.0
665,United Kingdom,UK,2019,32910.0,277548891.0,66647112.0
